### PyRosettaCluster 
## Tutorial 3. Multiple decoys

Tutorial 3 demonstrates various ways to efficiently run protocols multiple times. 

Parallelization can be accomplished by passing multiple tasks to PyRosettaCluster and/or PyRosettaCluster's `nstruct` argument. 

In addition, protocols can `yield` or `return` multiple `Pose` objects.

### 1. Import packages

In [1]:
import bz2
import glob
import json
import logging
logging.basicConfig(level=logging.INFO)
import os
import pyrosetta
import pyrosetta.distributed.io as io
import pyrosetta.distributed.packed_pose as packed_pose
import pyrosetta.distributed.tasks.rosetta_scripts as rosetta_scripts
import pyrosetta.distributed.tasks.score as score
import pyrosetta.distributed.viewer as viewer
import random
import tempfile

from pyrosettacluster import PyRosettaCluster, get_instance_kwargs, reproduce

### 2. Initialize a compute cluster using `dask`

1. Click the "Dask" tab in Jupyter Lab <i>(arrow, left)</i>
2. Click the "+ NEW" button to launch a new compute cluster <i>(arrow, lower)</i>

![title](images/dask_labextension_1.png)

3. Once the cluster has started, click the brackets to "inject client code" for the cluster into your notebook

![title](images/dask_labextension_2.png)

Inject client code here, then run the cell:

In [ ]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:40603")
client

Client Scheduler: tcp://127.0.0.1:38891 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 16.63 GB


### 3. Define the user-provided paths:

In [3]:
path_to_my_PyRosettaCluster_git_repo = '/shared/home/aloshbaugh/PyRosettaCluster'

in_dir = os.path.join( path_to_my_PyRosettaCluster_git_repo, 'tutorials/input' )
work_dir = os.path.join( path_to_my_PyRosettaCluster_git_repo, 'tutorials/3_Multiple_decoys' )

### 4. A protocol that returns multiple poses

PyRosettaCluster automatically passes poses through protocols supplied by the user. If a protocol produces `n` poses, the subsequent protocol runs `n` times, once for each pose. `Pose` objects returned by the final protocol are written to disk.

Multiple poses can be yielded iteratively, or returned as list or comma-separated:

Yield:

    for _ in range(n_results):
        yield backrub(ppose.pose.clone())

Return list:

    return list_of_poses

Return comma-separated:

    return pose1, pose2, pose3


In [13]:
def protocol1( packed_pose_in=None, **kwargs ):
    """
    Performs backrub on a pose, which can be (a) input to the function or (b) accessed through kwargs 's' argument.
    
    Args:
        ppose: A `PackedPose` object to be repacked. Optional.
        **kwargs: PyRosettaCluster keyword arguments.

    Returns:
        Multiple `PackedPose` objects.
    """
    import pyrosetta
    import pyrosetta.distributed.io as io
    import pyrosetta.distributed.tasks.rosetta_scripts as rosetta_scripts
    
    input_protocol = """
        <ROSETTASCRIPTS>
        
          <MOVERS>
            <Backrub name="backrub" pivot_residues="22A,23A,24A,25A,26A,27A" />
          </MOVERS>

          <PROTOCOLS>
            <Add mover="backrub"/>
          </PROTOCOLS>
        </ROSETTASCRIPTS>
        """
    backrub = rosetta_scripts.SingleoutputRosettaScriptsTask(input_protocol)
    
    if packed_pose_in == None:
        packed_pose_in = io.pose_from_file( kwargs['s'] )
    
    n_results = 3
    for _ in range(n_results):
        yield backrub(packed_pose_in.pose.clone())


def protocol2( packed_pose_in, **kwargs ):
    """
    Performs sequence design (Thr24 --> ALLAAxc) an input pose (Top7, pdb:1qys).
    
    Args:
        ppose: A `PackedPose` object to be designed.
        **kwargs: PyRosettaCluster keyword arguments.

    Returns:
        A `PackedPose` objects (Top7 T24 muts).
    """
    import pyrosetta
    import pyrosetta.distributed.tasks.rosetta_scripts as rosetta_scripts

    input_protocol = """
        <ROSETTASCRIPTS>

            <RESIDUE_SELECTORS>
                <Index name="T24" resnums="24A" />
                <Not name="not24" selector="T24" />
            </RESIDUE_SELECTORS>

            <TASKOPERATIONS>
                <ResfileCommandOperation name="T24_ALLAA" command="ALLAAxc" residue_selector="T24" />
                <OperateOnResidueSubset name="restrict_others" selector="not24" >
                    <PreventRepackingRLT/>
                </OperateOnResidueSubset>
            </TASKOPERATIONS>

            <MOVERS>
                <PackRotamersMover name="design_mover" task_operations="T24_ALLAA,restrict_others" />
            </MOVERS>

            <PROTOCOLS>
                <Add mover="design_mover"/>
            </PROTOCOLS>
        </ROSETTASCRIPTS>
        """
    design_protocol = rosetta_scripts.SingleoutputRosettaScriptsTask(input_protocol)
    
    packed_pose_out = design_protocol(packed_pose_in.pose.clone())
    
    return packed_pose_out

### 5. Define the user-provided kwargs:

In [14]:
def create_tasks():
    yield {
        "options": "-ex1",
        "extra_options": "-out:level 300 -multithreading:total_threads 1",
        "s":os.path.join( in_dir, '1QYS.pdb' ),
    }

def kwargs_for_tasks():
    yield 
    return[ 
        {
        "options": "-ex1",
        "extra_options":dictionary_of_options, # "-out:level 300 -multithreading:total_threads 1", 
        "s":os.path.join( in_dir, '1QYS.pdb' ),
        }, 
        {  "s":os.path.join( in_dir, '78fk.pdb' ), } 
    ] multiplies first protocol

### 6. Launch the original simulation using `distribute()`

The protocol produces a decoy, which we will reproduce at a later step.

In this example we use the `PyRosettaCluster` `nstruct` argument. `nstruct` is an `int` object specifying the number of repeats of the first user-provided PyRosetta protocol.

In [15]:
protocols = [ protocol1, protocol2, protocol1 ]

PyRosettaCluster(
    tasks=create_tasks,
    client=client,
    scratch_dir=work_dir,
    output_path=out_dir,
    nstruct=2
    
).distribute(protocols=protocols)

While jobs are running, you may monitor their progress using the dask dashboard diagnostics within Jupyter Lab!

In the "Dask" tab, click the various diagnostic tools _(arrows)_ to open new tabs:

![title](images/dask_labextension_4.png)

Arrange the diagnostic tool tabs within Jupyter Lab how you best see fit by clicking and dragging them:

![title](images/dask_labextension_3.png)

### 7. Visualize the resultant decoy

Gather pose from disk into memory:

In [10]:
results = glob.glob(os.path.join(out_dir, "decoys/*/*.pdb.bz2"))
packed_poses = []
for bz2file in results:
    with open(bz2file, "rb") as f:
        packed_poses.append(io.pose_from_pdbstring(bz2.decompress(f.read()).decode()))

View the pose in memory. 

Your designed Top7 (1qys) is shown in rainbow ribbon, with side chains in white sticks. The default view shows position 24 at top middle with blue ribbon. Backbone flexibility was modeled at positions 22-27, and position 24 was allowed to design to any amino acid except cysteine.

There are 6 result poses: 2 (nstruct) x 3 (protocol1) x 1 (protocol2)

Click and drag to rotate, zoom in and out w/ mouse scroll.

In [11]:
view = viewer.init(packed_poses, window_size=(800, 600))
view.add(viewer.setStyle())
view.add(viewer.setStyle(colorscheme="whiteCarbon", radius=0.25))
view.add(viewer.setHydrogenBonds())
view.add(viewer.setHydrogens(polar_only=True))
view.add(viewer.setDisulfides(radius=0.25))
view()

interactive(children=(IntSlider(value=0, continuous_update=False, description='Decoys', max=3), Output()), _do…

<function pyrosetta.distributed.viewer.core.Viewer.show.<locals>.view(i=0)>

### 8. Reproduce the decoy.

The protocol produced only one decoy, which is accessed by index zero of results:

In [165]:
decoy = results[0]

Optionally, `PyRosettaCluster` instance keyword arguments to reproduce this decoy can be viewed using `get_instance_kwargs()`.

In [166]:
instance_kwargs = get_instance_kwargs(input_file=decoy)
instance_kwargs

{'ami_id': '',
 'compressed': True,
 'cores': 1,
 'dashboard_address': ':8787',
 'decoy_dir_name': 'decoys',
 'decoy_ids': [0, 0, 0],
 'dry_run': False,
 'ignore_errors': False,
 'instance_id': '',
 'logging_level': 'INFO',
 'logs_dir_name': 'logs',
 'max_workers': 1000,
 'memory': '4g',
 'min_workers': 1,
 'nstruct': 1,
 'output_path': '/shared/home/aloshbaugh/PyRosettaCluster/tutorials/tutorial_3/output',
 'processes': 1,
 'project_name': '2020.05.12.16.58.02.577024',
 'protocols': ['protocol1', 'protocol2', 'protocol1'],
 'save_all': False,
 'scheduler': None,
 'scorefile_name': 'scores.json',
 'scratch_dir': '/shared/home/aloshbaugh/PyRosettaCluster/tutorials/tutorial_3',
 'seeds': ['-1941677680', '-985468093', '633191549'],
 'sha1': '',
 'simulation_name': '2020.05.12.16.58.02.577024',
 'tasks': {'options': '-ex1',
  'extra_options': '-out:level 300 -multithreading:total_threads 1',
  's': '/shared/home/aloshbaugh/PyRosettaCluster/tutorials/input/1QYS.pdb'},
 'timeout': 0.5}

### 9. Launch the reproduction simulation using `reproduce()`

Reproducing the decoy is accomplished with the `PyRosettaCluster reproduce()` method. This method requires the same variables you specified for the `distribute()` method: `input_packed_pose`, `client`, and `protocols`. Additionally, it requires the file to reproduce: `input_file`. The information from `instance_kwargs` is accessed under the hood by `reproduce()`.

In [167]:
reproduce(
    input_file=decoy,
    protocols=protocols,
    instance_kwargs={"client": client, "output_path":out_dir, }, # "input_packed_pose":input_packed_pose, },
)
# jason is updating this to client=client and input_packed_pose=input_packed_pose
#input pose is optional for FIRST method, it could be None and got from kwargs. for subsequent methods, input pose is output from previous method
# Note: 

### 10. Visualize the reproduced decoy

In [168]:
for bz2file in glob.glob(os.path.join(out_dir, "decoys/*/*.pdb.bz2")):
    if bz2file not in results:
        reproduced_result = bz2file
        break

with open(reproduced_result, "rb") as f:
    reproduced_packed_pose = io.pose_from_pdbstring(bz2.decompress(f.read()).decode())

In [169]:
view = viewer.init(reproduced_packed_pose, window_size=(800, 600))
view.add(viewer.setStyle())
view.add(viewer.setStyle(colorscheme="whiteCarbon", radius=0.25))
view.add(viewer.setHydrogenBonds())
view.add(viewer.setHydrogens(polar_only=True))
view.add(viewer.setDisulfides(radius=0.25))
view()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

### 11. Optionally, perform sanity checks to confirm that the reproduced pose is identical to the original:

PyRosetta trajectories are _deterministic_ depending on the input random number generated seed(s)!

In [170]:
original_pose = packed_poses[0].pose
reproduced_pose = reproduced_packed_pose.pose

#### Assert that the sequences are identical:

In [171]:
assert original_pose.sequence() == reproduced_pose.sequence()

#### Assert that the `total_score`s are identical:

In [172]:
scorefxn = pyrosetta.create_score_function("ref2015.wts")
assert scorefxn(original_pose) == scorefxn(reproduced_pose)

#### Assert that the C$_{\alpha}$–C$_{\alpha}$ root-mean-square deviation (RMSD) is `0.0` Å:

Note: There is no need to first superimpose the `original_pose` and `reproduced_pose` because they were both generated starting from the same `input_packed_pose`

In [173]:
assert pyrosetta.rosetta.core.scoring.CA_rmsd(original_pose, reproduced_pose) == 0.0

#### The reason the `original_pose` and `reproduced_pose` are identical is because the `seeds`, `decoy_ids`, and `protocols` attributes were identical in both `PyRosettaCluster` simulations:

In [174]:
for attribute in ["seeds", "decoy_ids", "protocols"]:
    print(get_instance_kwargs(results[0])[attribute])
    assert get_instance_kwargs(reproduced_result)[attribute] == get_instance_kwargs(results[0])[attribute]

['-1941677680', '-985468093', '633191549']
[0, 0, 0]
['protocol1', 'protocol2', 'protocol1']


### Congrats! 
You have successfully reproduced a multi-protocol Rosetta trajectory using `PyRosettaCluster`! This ends Tutorial 3.